In [1]:
import pandas as pd
import numpy as np
import json

path_exec = '/Users/arturarakelan/Downloads/Задание orion/exec_logs.log'
path_market = '/Users/arturarakelan/Downloads/Задание orion/market_logs.log'

In [2]:
log_data=open(path_market,'r')
result={}
i=0
for line in log_data:
    columns = line.split(', ') #or w/e you're delimiter/separator is
    data={}
    for j in range(len(columns)):
        if j == 0:
            key = columns[j].split(' ')[2]
            value=columns[j].split(' ')[4]
            data[key]=value
            result[i]=data
            key = columns[j].split(' ')[6]
            value=columns[j].split(' ')[7]
            data[key]=value
            result[i]=data
        elif j == 2:
            key = 'bid'
            x_place = columns[j].find("x")
            value = float(columns[j][columns[j].find("@")+1:x_place])
            data[key]=value
            result[i]=data
            key = 'ask'
            value = float(columns[j][x_place+1:columns[j].find("@", x_place)])
            data[key]=value
            result[i]=data
        key = columns[j].split(' ')[0]
        value=columns[j].split(' ')[1]
        data[key]=value
    result[i]=data
    i+=1
j=json.dumps(result)

df_market=pd.read_json(j, orient='index')

In [3]:
log_data=open(path_exec,'r')
result={}
i=0
for line in log_data:
    columns = line.split(', ') #or w/e you're delimiter/separator is
    data={}
    for j in range(len(columns)):
        if j == 0:
            key = columns[j].split(' ')[2]
            value=columns[j].split(' ')[4]
            data[key]=value
            result[i]=data
            key = columns[j].split(' ')[6]
            value=columns[j].split(' ')[7]
            data[key]=value
            result[i]=data
        key = columns[j].split(' ')[0]
        value=columns[j].split(' ')[1]
        data[key]=value
    result[i]=data
    i+=1
j=json.dumps(result)

df_exec=pd.read_json(j, orient='index')

In [4]:
df_market['mid_price'] = (df_market['bid'] + df_market['ask'])/2
df_market.head()

,trd,side,mLog:,price,bid,ask,traded,mid_price
0,1622505601191796224,-1,market,1108.9,1108.7,1108.8,volume,1108.75
1,1622505601222248192,-1,market,1108.7,1108.5,1108.6,volume,1108.55
2,1622505601364445696,-1,market,1108.5,1108.3,1108.6,volume,1108.45
3,1622505601405326080,1,market,1108.6,1108.3,1108.6,volume,1108.45
4,1622505601406994688,-1,market,1108.3,1108.2,1108.3,volume,1108.25


In [114]:
df_exec.head()

,exec,id,eLog:,side,price,traded,volume,delta_execsend,time,exec_time_5
0,1622505601222248192,281481419161600,order,1,1108.6,volume,left,30451968,1622505601252700160,1622505601222248197
1,1622505601424113408,562956395872256,order,-1,1108.4,volume,left,17118720,1622505601441232128,1622505601424113413
2,1622505601604667904,844431372582912,order,1,1108.3,volume,left,153866496,1622505601758534400,1622505601604667909
3,1622505602982201088,1125906349293568,order,-1,1108.4,volume,left,153371136,1622505603135572224,1622505602982201093
4,1622505603600579584,1407381326004224,order,1,1108.3,volume,left,618378496,1622505604218958080,1622505603600579589


In [5]:
df_exec['exec_5_sec'] = df_exec['exec'] + 5*(10**9)

In [6]:
df_exec['exec_max'] = 1 

In [7]:
a = np.unique(df_market['trd'])
a

array([1622505601191796224, 1622505601222248192, 1622505601364445696, ...,
       1627626549091604992, 1627626549092524800, 1627626549492660480])

In [8]:
for i in range(len(df_exec)):
    try:
        df_exec['exec_max'].iloc[i] = np.max(a[(a>=df_exec['exec'].iloc[i]) & (a<=df_exec['exec_5_sec'].iloc[i])])
    except:
        pass 

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
df = df_exec.merge(df_market, how='inner', left_on='exec_max', right_on='trd')
df.head()

,exec,id,eLog:,side_x,price_x,traded_x,volume,delta_execsend,exec_5_sec,exec_max,trd,side_y,mLog:,price_y,bid,ask,traded_y,mid_price
0,1622505601222248192,281481419161600,order,1,1108.6,volume,left,30451968,1622505606222248192,1622505605897538816,1622505605897538816,-1,market,1108.7,1108.6,1108.8,volume,1108.70
1,1622505601424113408,562956395872256,order,-1,1108.4,volume,left,17118720,1622505606424113408,1622505606417009152,1622505606417009152,1,market,1108.7,1108.6,1108.7,volume,1108.65
2,1622505601604667904,844431372582912,order,1,1108.3,volume,left,153866496,1622505606604667904,1622505606417009152,1622505606417009152,1,market,1108.7,1108.6,1108.7,volume,1108.65
3,1622505602982201088,1125906349293568,order,-1,1108.4,volume,left,153371136,1622505607982201088,1622505607938699264,1622505607938699264,-1,market,1108.5,1108.4,1108.7,volume,1108.55
4,1622505603600579584,1407381326004224,order,1,1108.3,volume,left,618378496,1622505608600579584,1622505608426427136,1622505608426427136,-1,market,1108.4,1108.3,1108.4,volume,1108.35


In [10]:
df['dmid5'] = (df['mid_price'] - df['price_x']) * df['side_x']

In [11]:
np.corrcoef(df['dmid5'], df['delta_execsend'])[0][1]

0.029727284476203673